## Предварительная обработка текстовых файлов 

После распознавания текстов отчетов их предстоит дополнительно обработать.

#### Загружаем библиотеки, создаем вспомогательную функцию для обработки текста, указываем директорию для сохранения файлов

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from matplotlib import rc

import csv
import os


In [0]:
INPUT_DIR ='/content/gdrive/My Drive/'
REPORTS_DIR = '/content/reports/'

try:
  os.mkdir(REPORTS_DIR)
except FileExistsError:
      print(REPORTS_DIR+' already exists')


In [0]:
def clean_df(file_r, df):

  # Считываем текстовый файл с отчетом в таблицу, одна строка таблицы - один абзац текста
  df = pd.read_csv(file_r, sep='\n',header=None, names=['Sentence'],quoting=csv.QUOTE_NONE) 

  # Вычисляем длину строк, оставляем те, длина которых превышает 3 слова
  df['len'] = df.Sentence.apply(lambda x: len(x.split()))
  df = df[df.len >=3]

  # Разбиваем абзацы на отдельные предложения
  new_df = pd.DataFrame(df.Sentence.str.split('.').tolist(), index=df.index).stack()
  new_df = new_df.reset_index()
  new_df = new_df.drop(columns=['level_0', 'level_1'])
  new_df.columns = ['Sentence']
  new_df['len'] = new_df.Sentence.apply(lambda x: len(x.split()))

  # Обрабатываем текст: переводим слова в нижний регистр, оставляем предложения длиной больше 5 слов, оставляем только альфанумерические символы
  new_df.Sentence = new_df.Sentence.apply(lambda x: x.lower())
  new_df = new_df[new_df.len >=5]
  new_df.Sentence = new_df.Sentence.apply(lambda x: ' '.join(x for x in x.split() if x.isalnum()))

  # Сохраняем обработанный текст отчета
  file_w = file_r.strip('.txt')
  new_df.to_csv(path_or_buf="/content/reports/{}.csv".format(file_w)) 

  return new_df

#### Загружаем локально сохраненный текстовый файл

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
from google.colab import files

uploaded = files.upload() 
filename = [x for x in uploaded.keys()][0]
output_name = filename.replace('.txt','.csv')
print('input: ',filename, 'output: ', output_name)

In [0]:
df = pd.read_csv(filename, sep='\n',header=None, names=['Sentence'], quoting=csv.QUOTE_NONE)

#### Обрабатываем и сохраняем текст

In [0]:
df = clean_df(file_r=filename, df=df)
df.head(2)


Проверяем наличие файла

In [0]:
%cd /content/reports
!ls 